In [8]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Parameters
DATASET_PATH = "F:\\citrus_dataset"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
LEARNING_RATE = 0.001
CLASS_NAMES = [
    "Citrus canker", "Citrus greening", "Citrus mealybugs", "Die back",
    "Foliage damaged", "Healthy leaf", "Powdery mildew", "Shot hole",
    "Spiny whitefly", "Yellow leaves"
]

# Data Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

train_gen = data_gen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_gen = data_gen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

# Load DenseNet121 base model with only 80 layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model = Model(inputs=base_model.input, outputs=base_model.layers[79].output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(CLASS_NAMES), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=[early_stopping]
)

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model with a lower learning rate
optimizer = Adam(learning_rate=LEARNING_RATE / 10)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping]
)

# Evaluate the model
val_gen.reset()
y_true = val_gen.classes
y_pred = model.predict(val_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=CLASS_NAMES))

accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

save_path = os.path.expanduser(r'C:/Users/abdju/OneDrive/Desktop/model/citrus_disease_model_densenet2.keras')
model.save(save_path)


Found 4689 images belonging to 10 classes.
Found 1170 images belonging to 10 classes.
Epoch 1/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 2100s 14s/step - accuracy: 0.1339 - loss: 2.2944 - val_accuracy: 0.1496 - val_loss: 2.2889
Epoch 2/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1251s 9s/step - accuracy: 0.1741 - loss: 2.2508 - val_accuracy: 0.1983 - val_loss: 2.2703
Epoch 3/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1300s 9s/step - accuracy: 0.1974 - loss: 2.2015 - val_accuracy: 0.2085 - val_loss: 2.2538
Epoch 4/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1244s 8s/step - accuracy: 0.1917 - loss: 2.1641 - val_accuracy: 0.2265 - val_loss: 2.2563
Epoch 5/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1218s 8s/step - accuracy: 0.2115 - loss: 2.1481 - val_accuracy: 0.1863 - val_loss: 2.2691
Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 2087s 14s/step - accuracy: 0.2404 - loss: 2.0639 - val_accuracy: 0.1239 - val_loss: 2.5749
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 2045s 14s/step - accuracy: 0.4512 - loss: 1.5963 - val_accuracy: 0.4675 - val_loss: 1.6359
Epoch 